In [2]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import pytz
import re


# Start the timer
start_time = time.time()

base_url = 'https://www.immobiliare.it/vendita-case/trieste-provincia/?criterio=rilevanza&pag={}'

def get_individual(url):
    req = requests.get(url).text
    soup = BeautifulSoup(req, 'html.parser')

    title = soup.find('div', class_='in-titleBlock__content').find('h1').text if soup.find('div', class_='in-titleBlock__content') else None
    title_text = title.split(',', 1) if title else [None]
    titolo = title_text[0].strip() if title_text[0] else None
    area = title_text[1].strip() if len(title_text) > 1 else None


    ul_tag = soup.find('ul', class_='nd-list nd-list--pipe in-feat in-feat--full in-feat__mainProperty in-landingDetail__mainFeatures')
    price_li = ul_tag.find('li', class_='nd-list__item in-feat__item in-feat__item--main in-detail__mainFeaturesPrice')
    price_li2 = ul_tag.find('li', class_='nd-list__item in-feat__item in-feat__item--main in-detail__mainFeaturesPrice in-detail__mainFeaturesPrice--interactive')



    locali_li = ul_tag.find(lambda tag: tag.name == 'li' and tag.get('aria-label') in ['locali', 'locale'])
    superficie_li = ul_tag.find('li', attrs={'aria-label': 'superficie'})
    bagni_li = ul_tag.find(lambda tag: tag.name == 'li' and tag.get('aria-label') in ['bagni', 'bagno'])
    piano_li = ul_tag.find('li', attrs={'aria-label': 'piano'})
    
    
    # Check if piano_li value is a number
    if piano_li and re.search(r'\d+', piano_li.text):
        piano = int(re.search(r'\d+', piano_li.text).group())
    else:
        dl_tag = soup.find('dl', class_='in-realEstateFeatures__list')
        dt_tags = dl_tag.find_all('dt', class_='in-realEstateFeatures__title')
        dd_tags = dl_tag.find_all('dd', class_='in-realEstateFeatures__value')
        
        # Look for the 'piano' value in dt_tags and return the corresponding dd value if it's a number
        for dt, dd in zip(dt_tags, dd_tags):
            if dt.text.strip() == 'piano' and re.search(r'\d+', dd.text):
                piano = int(re.search(r'\d+', dd.text).group())
                break
        else:
            # If 'piano' value is still not found, look for 'TOTALE PIANI EDIFICIO' value in dt_tags and return the corresponding dd value if it's a number
            for dt, dd in zip(dt_tags, dd_tags):
                if dt.text.strip() == 'totale piani edificio' and re.search(r'\d+', dd.text):
                    piano = int(re.search(r'\d+', dd.text).group())
                    break
            else:
                # If 'piano' and 'TOTALE PIANI EDIFICIO' values are not found or not numbers, set piano to None
                piano = None

    if price_li:
        price = price_li.text.strip().replace("€ ", "") 
    elif price_li2:
        price = price_li2.contents[0].strip().replace("€ ", "")
    else: 
        price = None
    
    
    locali = locali_li.text.strip() if locali_li else None
    superficie = superficie_li.text.strip().replace("m²", "") if superficie_li else None
    bagni = bagni_li.text.strip() if bagni_li else None


    # Extract description
    dettagli_tag = soup.find('div', class_='in-readAll')
    dettagli = dettagli_tag.text.strip() if dettagli_tag else None
    descrizione = dettagli.split('.')[0].strip() if dettagli else None

    div_tag = soup.find('div', class_='in-referent in-referent__withPhone') or soup.find('div', class_='in-referent')
    agenzia = div_tag.find('p').text.strip() if div_tag and div_tag.find('p') else None

    DB_features = {}
    dl_tags = soup.find_all('dl', class_='in-realEstateFeatures__list')
    for dl_tag in dl_tags:
        if dl_tag:
            dt_tags = dl_tag.find_all('dt', class_='in-realEstateFeatures__title')
            dd_tags = dl_tag.find_all('dd', class_='in-realEstateFeatures__value')
            for dt, dd in zip(dt_tags, dd_tags):
                dt_text = dt.text.strip().lower()
                if dt_text == 'altre caratteristiche':
                    span_tags = dd.find_all('span', class_='in-realEstateFeatures__tag nd-tag')
                    dd_values = [span.text.strip() for span in span_tags]
                    DB_features[dt_text] = ', '.join(dd_values)
                elif dt_text == 'efficienza energetica':
                    energy_class = dd.find('span', class_='in-realEstateFeatures__energy').text.strip()
                    DB_features[dt_text] = energy_class
                else:
                    DB_features[dt_text] = dd.text.strip()





    # Filtering out unwanted keys from altre_caratteristiche
    unwanted_keys = ['superficie', 'locali', 'prezzo', 'spese condominio', 'cauzione','piano']
    DB_features = {k: v for k, v in DB_features.items() if k not in unwanted_keys}

    #for the blueprint url
    bp_tag = soup.find('div', class_='nd-figure__image nd-ratio nd-ratio--wide in-landingDetail__simpleGallery')
    img_tag = bp_tag.find('img', class_='nd-figure__content nd-ratio__img') if bp_tag else None

    # One-liner if-else statement
    DB_features['Blueprint Url'] = img_tag.get('src', 'Not available') if img_tag else 'Not available'


    # Include the date and time the property was extracted
    extracted_datetime = datetime.now()

    # Get the GMT time zone
    gmt = pytz.timezone('GMT')

    # Convert the extracted_datetime to GMT time zone
    extracted_datetime_gmt = extracted_datetime.astimezone(gmt)

    all_features = {}



    GS_features = {
        'Date_GMT': extracted_datetime_gmt.strftime('%Y-%m-%d %H:%M:%S %Z'),
        'Titolo': titolo,
        'Area':area,
        'Link': url,
        'Prezzo': price,
        'Locali': locali,
        'MQ': superficie,
        'Bagni': bagni,
        'Piano':piano,
        'Descrizione': descrizione,
        'Dettagli': dettagli,
        'Agenzia': agenzia,

    }

    all_features.update(GS_features)
    all_features.update(DB_features)

    return all_features

def scrape_data():
    total_properties = 0
    total_pages = 0

    url = base_url.format(1)
    req = requests.get(url).text
    soup = BeautifulSoup(req, 'html.parser')

    total_properties = int(soup.find('div', class_='in-searchList__title').text.split()[0].replace('.', ''))
    total_pages = (total_properties + 24) // 25

    all_urls = []
    for page_number in range(1, total_pages + 1):
        url = base_url.format(page_number)
        req = requests.get(url).text
        soup = BeautifulSoup(req, 'html.parser')
        house_list = soup.find_all('div', class_='nd-mediaObject__content in-card__content in-realEstateListCard__content')

        for house in house_list:
            house_url = house.find('a')['href']
            all_urls.append(house_url)

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(get_individual, url) for url in all_urls]

    details_list = [future.result() for future in futures]
    df = pd.DataFrame(details_list)

    return df



# Scrape data and print the resulting DataFrame
dataframe = scrape_data()

#Getting the dataframe for the Google sheet
GS_columns = ['Date_GMT', 'Titolo', 'Area', 'Link', 'Prezzo', 'Locali', 'MQ', 'Bagni','Piano', 'Descrizione', 'Dettagli', 'Agenzia']
GS_df = dataframe[GS_columns]

#Geting the dataframe for the Database
DB_columns = ['Date_GMT', 'Titolo', 'Area', 'Link', 'Prezzo', 'Locali', 'MQ', 'Bagni','Piano', 'Descrizione', 'Dettagli', 'Agenzia',
              'contratto', 'anno di costruzione', 'stato', 'riscaldamento','efficienza energetica', 'altre caratteristiche','Blueprint Url']

DB_df = dataframe[DB_columns]


# Calculate the elapsed time
elapsed_time = time.time() - start_time

# Print the elapsed time
print(f"Elapsed time: {elapsed_time} seconds")



Elapsed time: 640.880040884018 seconds


In [3]:
dataframe.head()

,Date_GMT,Titolo,Area,Link,Prezzo,Locali,MQ,Bagni,Piano,Descrizione,...,numero immobili,categoria,aggiornato il,numero procedura,procedura,tribunale,referente,termine presentazione,giudice,curatore
0,2023-08-01 17:41:19 GMT,Trilocale via dei Piccardi 1,"Largo Barriera - Ospedale Maggiore, Trieste",https://www.immobiliare.it/annunci/104711337/,145.000,3,90,2,7.0,AD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-08-01 17:41:19 GMT,Trilocale via Petronio 3,"Settefontane, Trieste",https://www.immobiliare.it/annunci/104865777/,185.000,3,90,1,5.0,"VIA PETRONIO:In zona ben servita, a due passi ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-08-01 17:41:19 GMT,Trilocale via Giovanni Pascoli 16,"Largo Barriera - Ospedale Maggiore, Trieste",https://www.immobiliare.it/annunci/104996375/,158.000,3,100,2,1.0,VIA GIOVANNI PASCOLI:A breve distanza da Largo...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-08-01 17:41:19 GMT,Trilocale viale Miramare 23,"Roiano, Trieste",https://www.immobiliare.it/annunci/104849015/,270.000,3,135,1,3.0,PDM Real Estate propone in vendita questo spet...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-08-01 17:41:19 GMT,Bilocale Strada di Portopiccolo,Duino-Aurisina,https://www.immobiliare.it/annunci/98582788/,489.000,2,84,1,1.0,NUOVO APPARTAMENTO - Nel prestigioso complesso...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Auction Properties

In [23]:
auction_df = dataframe[dataframe['Prezzo'].str.contains('da', na=False)]
auction_df.head()

,Date_GMT,Titolo,Area,Link,Prezzo,Locali,MQ,Bagni,Piano,Descrizione,...,numero immobili,categoria,aggiornato il,numero procedura,procedura,tribunale,referente,termine presentazione,giudice,curatore
146,2023-08-01 17:42:16 GMT,Appartamento all'asta via Armando Diaz,"17, Trieste",https://www.immobiliare.it/annunci/104026875/,"da 360.000,00",5+,205,1,1.0,Licenza ai sensi dell’art,...,1,IMMOBILE RESIDENZIALE,19/06/2023,190/2018,Esecuzione Immobiliare,Trieste,NaN,NaN,NaN,NaN
150,2023-08-01 17:42:17 GMT,Appartamento all'asta via Armando Diaz,"17, Trieste",https://www.immobiliare.it/annunci/103681410/,"da 360.000,00",5+,205,1,1.0,Alloggio al primo piano dello stabile in Tries...,...,1,NaN,06/06/2023,190/2018,NaN,Trieste,NaN,NaN,NaN,NaN
153,2023-08-01 17:42:18 GMT,Villa all'asta via Vitalba,"3, Trieste",https://www.immobiliare.it/annunci/103594686/,"da 558.744,00",5+,217,1,2.0,Villa di mq 217 con giardino e quota pari ad 1...,...,1,NaN,01/06/2023,1489/2022,NaN,Trieste,NaN,NaN,NaN,NaN
286,2023-08-01 17:43:02 GMT,Appartamento all'asta via Armando Diaz 17,Trieste,https://www.immobiliare.it/annunci/103691064/,"da 360.000,00",5,205,1,1.0,Alloggio al primo piano dell'edificio in Tries...,...,1,IMMOBILE RESIDENZIALE,06/06/2023,2018,NaN,NaN,TRIESTE-06/10/2023-190/2018-Rita-1973351,"entro il 03/10/2023, 12:00",NaN,NaN
590,2023-08-01 17:44:42 GMT,Appartamento all'asta via Bellosguardo 52,Trieste,https://www.immobiliare.it/annunci/104979235/,"da 186.825,00",3,152,1,3.0,RIFEREMENTO: 2010231appartamento sito al terzo...,...,1,IMMOBILE RESIDENZIALE,24/07/2023,2022,NaN,NaN,TRIESTE-02/11/2023-3230/2022-Maria-2010231,NaN,NaN,NaN


In [24]:
GS_columns = ['Date_GMT', 'Titolo', 'Area', 'Link', 'Prezzo', 
              'Locali', 'MQ', 'Bagni','Piano', 'Descrizione', 'Dettagli', 'Agenzia']
auctionGS_df = auction_df[GS_columns]

#Geting the dataframe for the Database
DB_columns = ['Date_GMT', 'Titolo', 'Area', 'Link', 'Prezzo','Locali', 'MQ', 'Bagni','Piano', 
              'Descrizione', 'Dettagli', 'Agenzia','contratto', 'anno di costruzione', 'stato',
              'riscaldamento','efficienza energetica', 'altre caratteristiche','Blueprint Url']

auctionDB_df = auction_df[DB_columns]

In [25]:
auctionGS_df.head()

,Date_GMT,Titolo,Area,Link,Prezzo,Locali,MQ,Bagni,Piano,Descrizione,Dettagli,Agenzia
146,2023-08-01 17:42:16 GMT,Appartamento all'asta via Armando Diaz,"17, Trieste",https://www.immobiliare.it/annunci/104026875/,"da 360.000,00",5+,205,1,1.0,Licenza ai sensi dell’art,Licenza ai sensi dell’art. 115 TULPS rilasciat...,Asteadvisor Sede Centrale
150,2023-08-01 17:42:17 GMT,Appartamento all'asta via Armando Diaz,"17, Trieste",https://www.immobiliare.it/annunci/103681410/,"da 360.000,00",5+,205,1,1.0,Alloggio al primo piano dello stabile in Tries...,Alloggio al primo piano dello stabile in Tries...,Luffarelli Aste Immobiliari
153,2023-08-01 17:42:18 GMT,Villa all'asta via Vitalba,"3, Trieste",https://www.immobiliare.it/annunci/103594686/,"da 558.744,00",5+,217,1,2.0,Villa di mq 217 con giardino e quota pari ad 1...,Villa di mq 217 con giardino e quota pari ad 1...,Luffarelli Aste Immobiliari
286,2023-08-01 17:43:02 GMT,Appartamento all'asta via Armando Diaz 17,Trieste,https://www.immobiliare.it/annunci/103691064/,"da 360.000,00",5,205,1,1.0,Alloggio al primo piano dell'edificio in Tries...,Alloggio al primo piano dell'edificio in Tries...,Aste Florio
590,2023-08-01 17:44:42 GMT,Appartamento all'asta via Bellosguardo 52,Trieste,https://www.immobiliare.it/annunci/104979235/,"da 186.825,00",3,152,1,3.0,RIFEREMENTO: 2010231appartamento sito al terzo...,RIFEREMENTO: 2010231appartamento sito al terzo...,Aste Florio


In [26]:
auctionDB_df.head()

,Date_GMT,Titolo,Area,Link,Prezzo,Locali,MQ,Bagni,Piano,Descrizione,Dettagli,Agenzia,contratto,anno di costruzione,stato,riscaldamento,efficienza energetica,altre caratteristiche,Blueprint Url
146,2023-08-01 17:42:16 GMT,Appartamento all'asta via Armando Diaz,"17, Trieste",https://www.immobiliare.it/annunci/104026875/,"da 360.000,00",5+,205,1,1.0,Licenza ai sensi dell’art,Licenza ai sensi dell’art. 115 TULPS rilasciat...,Asteadvisor Sede Centrale,Vendita,1823,Partecipabile,"Centralizzato, a radiatori, alimentato a metano",G,"Cucina, Balcone, Impianto tv centralizzato, Es...",https://pic.im-cdn.it/plan/98607627/m.jpg
150,2023-08-01 17:42:17 GMT,Appartamento all'asta via Armando Diaz,"17, Trieste",https://www.immobiliare.it/annunci/103681410/,"da 360.000,00",5+,205,1,1.0,Alloggio al primo piano dello stabile in Tries...,Alloggio al primo piano dello stabile in Tries...,Luffarelli Aste Immobiliari,Vendita,1980,Buono / Abitabile,"Autonomo, a radiatori, alimentato a gas",F,"Esposizione esterna, Impianto di allarme, Vide...",https://pic.im-cdn.it/plan/98301406/m.jpg
153,2023-08-01 17:42:18 GMT,Villa all'asta via Vitalba,"3, Trieste",https://www.immobiliare.it/annunci/103594686/,"da 558.744,00",5+,217,1,2.0,Villa di mq 217 con giardino e quota pari ad 1...,Villa di mq 217 con giardino e quota pari ad 1...,Luffarelli Aste Immobiliari,Vendita,1980,Buono / Abitabile,"Autonomo, a radiatori, alimentato a gas",F,"Cancello elettrico, Fibra ottica, Porta blinda...",https://pic.im-cdn.it/plan/98227468/m.jpg
286,2023-08-01 17:43:02 GMT,Appartamento all'asta via Armando Diaz 17,Trieste,https://www.immobiliare.it/annunci/103691064/,"da 360.000,00",5,205,1,1.0,Alloggio al primo piano dell'edificio in Tries...,Alloggio al primo piano dell'edificio in Tries...,Aste Florio,Vendita,1967,Partecipabile,"Autonomo, a radiatori, alimentazione elettrica",G,"Impianto di allarme, Cancello elettrico, Camin...",https://pic.im-cdn.it/plan/98300148/m.jpg
590,2023-08-01 17:44:42 GMT,Appartamento all'asta via Bellosguardo 52,Trieste,https://www.immobiliare.it/annunci/104979235/,"da 186.825,00",3,152,1,3.0,RIFEREMENTO: 2010231appartamento sito al terzo...,RIFEREMENTO: 2010231appartamento sito al terzo...,Aste Florio,Vendita,NaN,Partecipabile,"Autonomo, a radiatori, alimentazione elettrica",G,"Impianto di allarme, Porta blindata, Armadio a...",https://pic.im-cdn.it/plan/99433171/m.jpg


### Building Properties

In [5]:
building_df = dataframe[dataframe['Prezzo'].str.contains(r'\d+\s*-\s*\d+', na=False)]
building_df

,Date_GMT,Titolo,Area,Link,Prezzo,Locali,MQ,Bagni,Piano,Descrizione,...,numero immobili,categoria,aggiornato il,numero procedura,procedura,tribunale,referente,termine presentazione,giudice,curatore
5,2023-08-01 17:41:19 GMT,Appartamenti e Attici di nuova costruzione a T...,None,https://www.immobiliare.it/annunci/99036760/,160.000 - 500.000,1 - 4,da57,None,4.0,Le Terrazze sul Golfo è il nuovo progetto resi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,2023-08-01 17:41:48 GMT,Appartamenti di nuova costruzione a Trieste,None,https://www.immobiliare.it/annunci/104853839/,170.000 - 595.000,1 - 4,da48,None,2.0,"In via Gambini a Trieste, a pochi passi da Lar...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,2023-08-01 17:42:05 GMT,Appartamenti e Attici di nuova costruzione a D...,None,https://www.immobiliare.it/annunci/98486664/,497.000 - 910.000,2 - 3,da95,None,6.0,"Scopri Portopiccolo, un incantevole borgo mari...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,2023-08-01 17:42:50 GMT,Attici e Appartamenti di nuova costruzione a T...,None,https://www.immobiliare.it/annunci/104126595/,360.000 - 1.520.000,2 - 4,da67,None,3.0,Romagna Alta è un progetto che coniuga innovaz...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
GS_columns = ['Date_GMT', 'Titolo', 'Area', 'Link', 'Prezzo', 
              'Locali', 'MQ', 'Bagni','Piano', 'Descrizione', 'Dettagli', 'Agenzia']
buildingGS_df = building_df[GS_columns]

#Geting the dataframe for the Database
DB_columns = ['Date_GMT', 'Titolo', 'Area', 'Link', 'Prezzo','Locali', 'MQ', 'Bagni','Piano', 
              'Descrizione', 'Dettagli', 'Agenzia','contratto', 'anno di costruzione', 'stato',
              'riscaldamento','efficienza energetica', 'altre caratteristiche','Blueprint Url']

buildingDB_df = building_df[DB_columns]

In [21]:
buildingGS_df.head()

,Date_GMT,Titolo,Area,Link,Prezzo,Locali,MQ,Bagni,Piano,Descrizione,Dettagli,Agenzia
5,2023-08-01 17:41:19 GMT,Appartamenti e Attici di nuova costruzione a T...,None,https://www.immobiliare.it/annunci/99036760/,160.000 - 500.000,1 - 4,da57,None,4.0,Le Terrazze sul Golfo è il nuovo progetto resi...,Le Terrazze sul Golfo è il nuovo progetto resi...,Terrazze sul Golfo
76,2023-08-01 17:41:48 GMT,Appartamenti di nuova costruzione a Trieste,None,https://www.immobiliare.it/annunci/104853839/,170.000 - 595.000,1 - 4,da48,None,2.0,"In via Gambini a Trieste, a pochi passi da Lar...","In via Gambini a Trieste, a pochi passi da Lar...",Andrea Starc
115,2023-08-01 17:42:05 GMT,Appartamenti e Attici di nuova costruzione a D...,None,https://www.immobiliare.it/annunci/98486664/,497.000 - 910.000,2 - 3,da95,None,6.0,"Scopri Portopiccolo, un incantevole borgo mari...","Scopri Portopiccolo, un incantevole borgo mari...",PORTO PICCOLO
244,2023-08-01 17:42:50 GMT,Attici e Appartamenti di nuova costruzione a T...,None,https://www.immobiliare.it/annunci/104126595/,360.000 - 1.520.000,2 - 4,da67,None,3.0,Romagna Alta è un progetto che coniuga innovaz...,Romagna Alta è un progetto che coniuga innovaz...,TIRABORA CANTIERI


In [22]:
buildingDB_df.head()

,Date_GMT,Titolo,Area,Link,Prezzo,Locali,MQ,Bagni,Piano,Descrizione,Dettagli,Agenzia,contratto,anno di costruzione,stato,riscaldamento,efficienza energetica,altre caratteristiche,Blueprint Url
5,2023-08-01 17:41:19 GMT,Appartamenti e Attici di nuova costruzione a T...,None,https://www.immobiliare.it/annunci/99036760/,160.000 - 500.000,1 - 4,da57,None,4.0,Le Terrazze sul Golfo è il nuovo progetto resi...,Le Terrazze sul Golfo è il nuovo progetto resi...,Terrazze sul Golfo,Vendita,NaN,NaN,"Centralizzato, a pavimento, alimentato a pompa...",NaN,"Parcheggio bici, Giardino comune, Infissi este...",Not available
76,2023-08-01 17:41:48 GMT,Appartamenti di nuova costruzione a Trieste,None,https://www.immobiliare.it/annunci/104853839/,170.000 - 595.000,1 - 4,da48,None,2.0,"In via Gambini a Trieste, a pochi passi da Lar...","In via Gambini a Trieste, a pochi passi da Lar...",Andrea Starc,Vendita,NaN,NaN,"Centralizzato, a radiatori, alimentato a pompa...",NaN,Parcheggio bici,Not available
115,2023-08-01 17:42:05 GMT,Appartamenti e Attici di nuova costruzione a D...,None,https://www.immobiliare.it/annunci/98486664/,497.000 - 910.000,2 - 3,da95,None,6.0,"Scopri Portopiccolo, un incantevole borgo mari...","Scopri Portopiccolo, un incantevole borgo mari...",PORTO PICCOLO,Vendita - Scarica capitolato,NaN,NaN,"Centralizzato, a pavimento, alimentato a pompa...",A,"Giardino comune, Infissi esterni in doppio vet...",Not available
244,2023-08-01 17:42:50 GMT,Attici e Appartamenti di nuova costruzione a T...,None,https://www.immobiliare.it/annunci/104126595/,360.000 - 1.520.000,2 - 4,da67,None,3.0,Romagna Alta è un progetto che coniuga innovaz...,Romagna Alta è un progetto che coniuga innovaz...,TIRABORA CANTIERI,Vendita - Scarica capitolato,NaN,NaN,"Centralizzato, a pavimento",A1,NaN,Not available


### Normal Main Properties

In [6]:
main_df = dataframe[~dataframe['Prezzo'].str.contains(r'\d+\s*-\s*\d+|da', na=False)]
main_df.head()

,Date_GMT,Titolo,Area,Link,Prezzo,Locali,MQ,Bagni,Piano,Descrizione,...,numero immobili,categoria,aggiornato il,numero procedura,procedura,tribunale,referente,termine presentazione,giudice,curatore
0,2023-08-01 17:41:19 GMT,Trilocale via dei Piccardi 1,"Largo Barriera - Ospedale Maggiore, Trieste",https://www.immobiliare.it/annunci/104711337/,145.000,3,90,2,7.0,AD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-08-01 17:41:19 GMT,Trilocale via Petronio 3,"Settefontane, Trieste",https://www.immobiliare.it/annunci/104865777/,185.000,3,90,1,5.0,"VIA PETRONIO:In zona ben servita, a due passi ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-08-01 17:41:19 GMT,Trilocale via Giovanni Pascoli 16,"Largo Barriera - Ospedale Maggiore, Trieste",https://www.immobiliare.it/annunci/104996375/,158.000,3,100,2,1.0,VIA GIOVANNI PASCOLI:A breve distanza da Largo...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-08-01 17:41:19 GMT,Trilocale viale Miramare 23,"Roiano, Trieste",https://www.immobiliare.it/annunci/104849015/,270.000,3,135,1,3.0,PDM Real Estate propone in vendita questo spet...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-08-01 17:41:19 GMT,Bilocale Strada di Portopiccolo,Duino-Aurisina,https://www.immobiliare.it/annunci/98582788/,489.000,2,84,1,1.0,NUOVO APPARTAMENTO - Nel prestigioso complesso...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
main_df['Prezzo'] = pd.to_numeric(main_df['Prezzo'],errors = 'coerce')
main_df['MQ'] = pd.to_numeric(main_df['MQ'],errors = 'coerce')

main_df['Prezzo_al_mq'] = main_df['Prezzo']/main_df['MQ']

main_df.insert(5, 'Prezzo_al_mq', main_df.pop('Prezzo_al_mq'))
main_df.head()

C:\Users\OIA2EC~1.ORI\AppData\Local\Temp/ipykernel_1776/248393505.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['Prezzo'] = pd.to_numeric(main_df['Prezzo'],errors = 'coerce')
C:\Users\OIA2EC~1.ORI\AppData\Local\Temp/ipykernel_1776/248393505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['MQ'] = pd.to_numeric(main_df['MQ'],errors = 'coerce')
C:\Users\OIA2EC~1.ORI\AppData\Local\Temp/ipykernel_1776/248393505.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

,Date_GMT,Titolo,Area,Link,Prezzo,Prezzo_al_mq,Locali,MQ,Bagni,Piano,...,numero immobili,categoria,aggiornato il,numero procedura,procedura,tribunale,referente,termine presentazione,giudice,curatore
0,2023-08-01 17:41:19 GMT,Trilocale via dei Piccardi 1,"Largo Barriera - Ospedale Maggiore, Trieste",https://www.immobiliare.it/annunci/104711337/,145.0,1.611111,3,90.0,2,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-08-01 17:41:19 GMT,Trilocale via Petronio 3,"Settefontane, Trieste",https://www.immobiliare.it/annunci/104865777/,185.0,2.055556,3,90.0,1,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-08-01 17:41:19 GMT,Trilocale via Giovanni Pascoli 16,"Largo Barriera - Ospedale Maggiore, Trieste",https://www.immobiliare.it/annunci/104996375/,158.0,1.580000,3,100.0,2,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-08-01 17:41:19 GMT,Trilocale viale Miramare 23,"Roiano, Trieste",https://www.immobiliare.it/annunci/104849015/,270.0,2.000000,3,135.0,1,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-08-01 17:41:19 GMT,Bilocale Strada di Portopiccolo,Duino-Aurisina,https://www.immobiliare.it/annunci/98582788/,489.0,5.821429,2,84.0,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
GS_columns = ['Date_GMT', 'Titolo', 'Area', 'Link', 'Prezzo', 
              'Locali', 'MQ', 'Bagni','Piano', 'Descrizione', 'Dettagli', 'Agenzia']
mainGS_df = main_df[GS_columns]

#Geting the dataframe for the Database
DB_columns = ['Date_GMT', 'Titolo', 'Area', 'Link', 'Prezzo', 'Prezzo_al_mq','Locali', 'MQ', 'Bagni','Piano', 
              'Descrizione', 'Dettagli', 'Agenzia','contratto', 'anno di costruzione', 'stato',
              'riscaldamento','efficienza energetica', 'altre caratteristiche','Blueprint Url']

mainDB_df = main_df[DB_columns]


In [16]:
mainGS_df.head()

,Date_GMT,Titolo,Area,Link,Prezzo,Locali,MQ,Bagni,Piano,Descrizione,Dettagli,Agenzia
0,2023-08-01 17:41:19 GMT,Trilocale via dei Piccardi 1,"Largo Barriera - Ospedale Maggiore, Trieste",https://www.immobiliare.it/annunci/104711337/,145.0,3,90.0,2,7.0,AD,AD.ZE VIA ROSSETTI:Nelle immediate adiacenze d...,Affiliato Tecnocasa: IMMOBILIARE 7F SRL
1,2023-08-01 17:41:19 GMT,Trilocale via Petronio 3,"Settefontane, Trieste",https://www.immobiliare.it/annunci/104865777/,185.0,3,90.0,1,5.0,"VIA PETRONIO:In zona ben servita, a due passi ...","VIA PETRONIO:In zona ben servita, a due passi ...",Affiliato Tecnocasa: IMMOBILIARE 7F SRL
2,2023-08-01 17:41:19 GMT,Trilocale via Giovanni Pascoli 16,"Largo Barriera - Ospedale Maggiore, Trieste",https://www.immobiliare.it/annunci/104996375/,158.0,3,100.0,2,1.0,VIA GIOVANNI PASCOLI:A breve distanza da Largo...,VIA GIOVANNI PASCOLI:A breve distanza da Largo...,Affiliato Tecnocasa: IMMOBILIARE 7F SRL
3,2023-08-01 17:41:19 GMT,Trilocale viale Miramare 23,"Roiano, Trieste",https://www.immobiliare.it/annunci/104849015/,270.0,3,135.0,1,3.0,PDM Real Estate propone in vendita questo spet...,PDM Real Estate propone in vendita questo spet...,SPUNTON & PARTNERS
4,2023-08-01 17:41:19 GMT,Bilocale Strada di Portopiccolo,Duino-Aurisina,https://www.immobiliare.it/annunci/98582788/,489.0,2,84.0,1,1.0,NUOVO APPARTAMENTO - Nel prestigioso complesso...,NUOVO APPARTAMENTO - Nel prestigioso complesso...,PORTO PICCOLO


In [17]:
mainDB_df.head()

,Date_GMT,Titolo,Area,Link,Prezzo,Prezzo_al_mq,Locali,MQ,Bagni,Piano,Descrizione,Dettagli,Agenzia,contratto,anno di costruzione,stato,riscaldamento,efficienza energetica,altre caratteristiche,Blueprint Url
0,2023-08-01 17:41:19 GMT,Trilocale via dei Piccardi 1,"Largo Barriera - Ospedale Maggiore, Trieste",https://www.immobiliare.it/annunci/104711337/,145.0,1.611111,3,90.0,2,7.0,AD,AD.ZE VIA ROSSETTI:Nelle immediate adiacenze d...,Affiliato Tecnocasa: IMMOBILIARE 7F SRL,Vendita,1958,Buono / Abitabile,"Centralizzato, a radiatori, alimentato a metano",G,"Balcone, Terrazza, Cantina",Not available
1,2023-08-01 17:41:19 GMT,Trilocale via Petronio 3,"Settefontane, Trieste",https://www.immobiliare.it/annunci/104865777/,185.0,2.055556,3,90.0,1,5.0,"VIA PETRONIO:In zona ben servita, a due passi ...","VIA PETRONIO:In zona ben servita, a due passi ...",Affiliato Tecnocasa: IMMOBILIARE 7F SRL,Vendita,1961,Ottimo / Ristrutturato,"Centralizzato, a radiatori, alimentato a metano",G,"Balcone, Parzialmente Arredato, Cantina",Not available
2,2023-08-01 17:41:19 GMT,Trilocale via Giovanni Pascoli 16,"Largo Barriera - Ospedale Maggiore, Trieste",https://www.immobiliare.it/annunci/104996375/,158.0,1.580000,3,100.0,2,1.0,VIA GIOVANNI PASCOLI:A breve distanza da Largo...,VIA GIOVANNI PASCOLI:A breve distanza da Largo...,Affiliato Tecnocasa: IMMOBILIARE 7F SRL,Vendita,1968,Buono / Abitabile,"Centralizzato, a radiatori, alimentato a metano",G,Arredato,Not available
3,2023-08-01 17:41:19 GMT,Trilocale viale Miramare 23,"Roiano, Trieste",https://www.immobiliare.it/annunci/104849015/,270.0,2.000000,3,135.0,1,3.0,PDM Real Estate propone in vendita questo spet...,PDM Real Estate propone in vendita questo spet...,SPUNTON & PARTNERS,Vendita,1905,Da ristrutturare,"Autonomo, a radiatori, alimentato a metano",F,"Esposizione esterna, Balcone, Impianto tv sing...",https://pic.im-cdn.it/plan/99318099/m.jpg
4,2023-08-01 17:41:19 GMT,Bilocale Strada di Portopiccolo,Duino-Aurisina,https://www.immobiliare.it/annunci/98582788/,489.0,5.821429,2,84.0,1,1.0,NUOVO APPARTAMENTO - Nel prestigioso complesso...,NUOVO APPARTAMENTO - Nel prestigioso complesso...,PORTO PICCOLO,Vendita - Scarica capitolato,2014,Nuovo / In costruzione,"Centralizzato, a pavimento, alimentato a pompa...",A,"Cancello elettrico, Fibra ottica, Porta blinda...",https://pic.im-cdn.it/plan/99253501/m.jpg
